# Лабораторная работа 1. Извлечение признаков из текстовых данных


## Задание
- Реализовать алгоритм получения контекстных эмбеддингов (векторных представлений) слов.

- Реализовать один из алгоритмов CBoW или SkipGram для получения собственных моделей Word2Vec для выбранных данных.

- Визуализировать векторные представления в малой размерности.



## Ход работы
1. Загрузить коллекцию текстовых документов: просьбы жильцов. Можете использовать не весь датасет (подмножество строк таблицы). В этой ЛР вас интересует столбец с текстом обращений жильцов. Можете загрузить любой датасет для классификации текстовых данных.

2. Рассмотреть каждый текстовый документ отдельно. Выполнить предварительную обработку текстовых данных (как в ЛР 8 по ML). Токенизация, лемматизация (стемминг), удаление стоп-слов.

3. Составить матрицу контекстных эмбеддингов слов.

4. С помощью PCA уменьшить размер контекстных эмбеддингов (размер задать самостоятельно).

5. С помощью PyTorch реализовать один из алгоритмов Word2Vec: CBoW или SkipGram. Получить векторные представления текстов с помощью собственной реализации одного из этих алгоритмов.

6. Сохранить полученные различными способами векторные представления в файлы tsv. Визуализировать их с помощью сервиса Projector от авторов TensorFlow.

In [1]:
import pandas as pd
import torch
from torch import nn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from pymorphy3 import MorphAnalyzer
import string
from gensim.models import Word2Vec
import numpy as np
from sklearn.decomposition import PCA
from torch.utils.data import TensorDataset, DataLoader

### 1. Загрузить коллекцию текстовых документов: просьбы жильцов. Можете использовать не весь датасет (подмножество строк таблицы). В этой ЛР вас интересует столбец с текстом обращений жильцов. Можете загрузить любой датасет для классификации текстовых данных.

In [2]:
data = pd.read_csv("../../data/Petitions.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59889 entries, 0 to 59888
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    59889 non-null  int64 
 1   public_petition_text  59889 non-null  object
 2   reason_category       59889 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [4]:
data.head(1000)

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство
...,...,...,...
995,3279475,Повреждения дорожного покрытия проезжей части,Благоустройство
996,3356222,дворник убрал песок у подъезда и высыпал его н...,Благоустройство
997,3276541,Нужно заделать ямы во дворе,Благоустройство
998,3032938,демонтировать решетку,Нарушение правил пользования общим имуществом


In [5]:
data = data.head(10000)

In [6]:
data.drop(["id", "reason_category"], axis=1, inplace=True)

### 2. Рассмотреть каждый текстовый документ отдельно. Выполнить предварительную обработку текстовых данных (как в ЛР 8 по ML). Токенизация, лемматизация (стемминг), удаление стоп-слов.

Удаление всех символов, которые не являются цифрами, буквами и пробелами, и токенизация

In [7]:
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('punkt_tab')

In [8]:
morph = MorphAnalyzer()
stopwords = stopwords.words("russian")


def preProccesingData(text):
    only_letters = re.sub("[^\s^\w]+", " ", text)
    tokenized = word_tokenize(only_letters)
    normalized = [morph.normal_forms(word)[0] for word in tokenized]
    no_stop_words = [
        word
        for word in normalized
        if ((word not in string.punctuation) and (word not in stopwords) and word.isalpha())
    ]
    return no_stop_words

In [9]:
data = data['public_petition_text'].apply(preProccesingData).tolist()
data[:10]

[['снег', 'дорога'],
 ['очистить', 'кабельный', 'киоск', 'реклама'],
 ['просить',
  'убрать',
  'всё',
  'дерево',
  'кустарник',
  'который',
  'выйти',
  'предел',
  'газон',
  'пешеходный',
  'зона',
  'начинать',
  'подъезд',
  'подъезд',
  'фасад',
  'дом',
  'сторона',
  'ул',
  'наличный'],
 ['неудовлетворительный', 'состояние', 'парадный', 'надпись', 'дверь', 'этаж'],
 ['граффити'],
 ['необходимо',
  'проверить',
  'законность',
  'установка',
  'вывеска',
  'фасад',
  'мкд',
  'адрес',
  'проспект',
  'непокорённый',
  'случай',
  'вывеска',
  'установить',
  'незаконно',
  'её',
  'необходимо',
  'демонтировать'],
 ['уборка',
  'производиться',
  'лестница',
  'очень',
  'грязно',
  'весь',
  'этаж',
  'вплоть',
  'го',
  'звонок',
  'жкс',
  'дать',
  'результат'],
 ['мусор'],
 ['отсутствовать', 'освещение', 'лестничный', 'площадка', 'этаж', 'парадный'],
 ['делать', 'благоустройство', 'никто', 'убирать', 'мусор', 'ежедневно']]

### 3. Составить матрицу контекстных эмбеддингов слов.

In [10]:
all_words = [word for document in data for word in document]

In [11]:
all_words[:10]

['снег',
 'дорога',
 'очистить',
 'кабельный',
 'киоск',
 'реклама',
 'просить',
 'убрать',
 'всё',
 'дерево']

In [12]:
vocab = set(all_words)
vocab_len = len(vocab)
vocab_len

7771

In [13]:
id_word = dict(enumerate(vocab))
id_word

{0: 'отломать',
 1: 'строиться',
 2: 'издевательство',
 3: 'пара',
 4: 'промежуточный',
 5: 'вместе',
 6: 'вывозить',
 7: 'шурф',
 8: 'предприниматься',
 9: 'огорождение',
 10: 'добрго',
 11: 'малоохтинский',
 12: 'песочница',
 13: 'лужие',
 14: 'материальный',
 15: 'быстро',
 16: 'время',
 17: 'колёрный',
 18: 'нарушеннго',
 19: 'взудитие',
 20: 'реальность',
 21: 'прозрачный',
 22: 'стена',
 23: 'наверху',
 24: 'комар',
 25: 'выступать',
 26: 'результативный',
 27: 'сфетофора',
 28: 'коляска',
 29: 'готовый',
 30: 'лом',
 31: 'посредством',
 32: 'выбоина',
 33: 'элемент',
 34: 'оно',
 35: 'планировочный',
 36: 'недостоверный',
 37: 'пётр',
 38: 'наклеить',
 39: 'недомыть',
 40: 'гнилостный',
 41: 'согласовать',
 42: 'сердобольский',
 43: 'первомайский',
 44: 'деревенский',
 45: 'срок',
 46: 'свыше',
 47: 'импровизированный',
 48: 'дезинфицировать',
 49: 'справа',
 50: 'разобрать',
 51: 'физически',
 52: 'примыкать',
 53: 'подводный',
 54: 'эвакуационный',
 55: 'повсеместно',
 56: 'ос

In [14]:
pd.DataFrame(data=id_word, columns=["id", "word"]).to_csv("id_word.tsv", index=False)

In [15]:
word_id = {v: k for k, v in id_word.items()}
word_id

{'отломать': 0,
 'строиться': 1,
 'издевательство': 2,
 'пара': 3,
 'промежуточный': 4,
 'вместе': 5,
 'вывозить': 6,
 'шурф': 7,
 'предприниматься': 8,
 'огорождение': 9,
 'добрго': 10,
 'малоохтинский': 11,
 'песочница': 12,
 'лужие': 13,
 'материальный': 14,
 'быстро': 15,
 'время': 16,
 'колёрный': 17,
 'нарушеннго': 18,
 'взудитие': 19,
 'реальность': 20,
 'прозрачный': 21,
 'стена': 22,
 'наверху': 23,
 'комар': 24,
 'выступать': 25,
 'результативный': 26,
 'сфетофора': 27,
 'коляска': 28,
 'готовый': 29,
 'лом': 30,
 'посредством': 31,
 'выбоина': 32,
 'элемент': 33,
 'оно': 34,
 'планировочный': 35,
 'недостоверный': 36,
 'пётр': 37,
 'наклеить': 38,
 'недомыть': 39,
 'гнилостный': 40,
 'согласовать': 41,
 'сердобольский': 42,
 'первомайский': 43,
 'деревенский': 44,
 'срок': 45,
 'свыше': 46,
 'импровизированный': 47,
 'дезинфицировать': 48,
 'справа': 49,
 'разобрать': 50,
 'физически': 51,
 'примыкать': 52,
 'подводный': 53,
 'эвакуационный': 54,
 'повсеместно': 55,
 'освещё

In [16]:
matrix_context = np.zeros((vocab_len, vocab_len))
window_size = 2

for doc in data:
    for i, word_one in enumerate(doc):
        for word_two in doc[i - window_size : i + window_size + 1]:
            if word_two != word_one:
                matrix_context[word_id[word_one], word_id[word_two]] += 1
            

In [17]:
list(pd.DataFrame(all_words)[0])

['снег',
 'дорога',
 'очистить',
 'кабельный',
 'киоск',
 'реклама',
 'просить',
 'убрать',
 'всё',
 'дерево',
 'кустарник',
 'который',
 'выйти',
 'предел',
 'газон',
 'пешеходный',
 'зона',
 'начинать',
 'подъезд',
 'подъезд',
 'фасад',
 'дом',
 'сторона',
 'ул',
 'наличный',
 'неудовлетворительный',
 'состояние',
 'парадный',
 'надпись',
 'дверь',
 'этаж',
 'граффити',
 'необходимо',
 'проверить',
 'законность',
 'установка',
 'вывеска',
 'фасад',
 'мкд',
 'адрес',
 'проспект',
 'непокорённый',
 'случай',
 'вывеска',
 'установить',
 'незаконно',
 'её',
 'необходимо',
 'демонтировать',
 'уборка',
 'производиться',
 'лестница',
 'очень',
 'грязно',
 'весь',
 'этаж',
 'вплоть',
 'го',
 'звонок',
 'жкс',
 'дать',
 'результат',
 'мусор',
 'отсутствовать',
 'освещение',
 'лестничный',
 'площадка',
 'этаж',
 'парадный',
 'делать',
 'благоустройство',
 'никто',
 'убирать',
 'мусор',
 'ежедневно',
 'просьба',
 'закрасить',
 'реклама',
 'забор',
 'снег',
 'тротуар',
 'убрать',
 'проблема',
 '

### 4. С помощью PCA уменьшить размер контекстных эмбеддингов (размер задать самостоятельно).

In [18]:
matrix_context_pca = PCA(n_components=3).fit_transform(matrix_context)

In [19]:
matrix_context_pca_round = np.round(matrix_context_pca, 5)

In [20]:
matrix_context_with_word = dict()

for i, word in id_word.items():
    matrix_context_with_word[word] = matrix_context_pca[i]
    
matrix_context_with_word

{'отломать': array([-1.44553453,  0.62872399, -0.02597886]),
 'строиться': array([-1.50660367,  0.30338148, -0.08694989]),
 'издевательство': array([-1.52272705,  0.44373145, -0.12269015]),
 'пара': array([ 7.44616998, -6.99959938,  4.40037903]),
 'промежуточный': array([-0.78875216,  0.98021995,  0.18064898]),
 'вместе': array([-0.86855884, -0.27485124,  0.23199031]),
 'вывозить': array([-0.0225091 , -1.23262574, -0.42693644]),
 'шурф': array([-1.55199799,  0.35615122, -0.10110165]),
 'предприниматься': array([-1.62286778,  0.40371952, -0.12079827]),
 'огорождение': array([-1.44647844,  0.59977447, -0.02518229]),
 'добрго': array([-1.65630525,  0.43927732, -0.10478637]),
 'малоохтинский': array([-0.92839808,  0.27829357, -0.30479659]),
 'песочница': array([-0.72648198, -0.59184534, -0.21725171]),
 'лужие': array([-1.48009783,  0.06318099, -0.24140961]),
 'материальный': array([-1.62919524,  0.42054926, -0.11245925]),
 'быстро': array([-1.55337765,  0.31533121, -0.13597353]),
 'время':

In [21]:
np.savetxt("matrix_context_pca.tsv", matrix_context_pca)

### 5. С помощью PyTorch реализовать один из алгоритмов Word2Vec: CBoW или SkipGram. Получить векторные представления текстов с помощью собственной реализации одного из этих алгоритмов.

In [22]:
X = []
y = []
max_id = len(id_word)
max_content = window_size * 2

for doc in data:
    doc_len = len(doc)
    for i, word in enumerate(doc):
        if doc_len == 1: word_context = [word]
        else: word_context = doc[max(0, i - window_size):i] + doc[i + 1: i + window_size + 1]

        X.append([word_id[w] for w in word_context])
        y.append(word_id[word])
        
        if (max_content - len(word_context)) % max_content > 0:
            X[-1] += [max_id] * ((max_content - len(word_context)) % max_content)

In [23]:
torch.tensor(X).size(), torch.tensor(y).size()

(torch.Size([100224, 4]), torch.Size([100224]))

In [24]:
class CBoW(nn.Module):
    def __init__(self, num_embeddings, embendding_dim):
        super(CBoW, self).__init__()
        self.embedding = nn.Embedding(
            num_embeddings=num_embeddings, embedding_dim=embendding_dim
        )
        self.linear = nn.Linear(embendding_dim, num_embeddings)

    def forward(self, x):
        y = self.embedding(x).mean(1)
        y = self.linear(y)
        return y

In [25]:
model = CBoW(vocab_len + 1, 35)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [26]:
X_tensor = torch.LongTensor(X)
y_tensor = torch.LongTensor(y)

train_ds = TensorDataset(X_tensor, y_tensor)
train_dl = DataLoader(train_ds, batch_size=1024,shuffle=True)

In [27]:
epochs = 50
for epoch in range(epochs):
    for x, y in train_dl:
        outputs = model(x)
        loss_value = loss(outputs, y)
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}")

Эпоха 1, Значение функции потерь: 8.97099494934082
Эпоха 2, Значение функции потерь: 8.925679206848145
Эпоха 3, Значение функции потерь: 8.866296768188477
Эпоха 4, Значение функции потерь: 8.817401885986328
Эпоха 5, Значение функции потерь: 8.764063835144043
Эпоха 6, Значение функции потерь: 8.692663192749023
Эпоха 7, Значение функции потерь: 8.684788703918457
Эпоха 8, Значение функции потерь: 8.60008430480957
Эпоха 9, Значение функции потерь: 8.564786911010742
Эпоха 10, Значение функции потерь: 8.479599952697754
Эпоха 11, Значение функции потерь: 8.394320487976074
Эпоха 12, Значение функции потерь: 8.32935619354248
Эпоха 13, Значение функции потерь: 8.336527824401855
Эпоха 14, Значение функции потерь: 8.25261116027832
Эпоха 15, Значение функции потерь: 8.194457054138184
Эпоха 16, Значение функции потерь: 8.107675552368164
Эпоха 17, Значение функции потерь: 7.993314266204834
Эпоха 18, Значение функции потерь: 7.978990077972412
Эпоха 19, Значение функции потерь: 7.910995960235596
Эпоха 

In [28]:
weight = model.linear.weight.detach().numpy()

In [32]:
weight.shape

(7772, 35)

In [29]:
len(weight)

7772

In [30]:
id_word[7300], weight[7300]

('осадка',
 array([-0.12710916,  0.20813441,  0.18532494, -0.07366374,  0.02282931,
        -0.12198538, -0.05412217, -0.13222803,  0.09234834,  0.17150405,
         0.05122293,  0.21620837,  0.20281482,  0.13229434,  0.02995811,
        -0.302809  ,  0.12605064,  0.10542149, -0.12809323, -0.27906603,
        -0.14581719, -0.12399022,  0.02439718,  0.15829265, -0.01993027,
         0.02203304,  0.16325116,  0.15210249, -0.1269648 , -0.18010657,
         0.13687168, -0.20311813,  0.2225621 , -0.05465198,  0.1223396 ],
       dtype=float32))

In [33]:
np.savetxt("cbow_emb_pca.tsv", PCA(10).fit_transform(weight))